In [179]:
from typing import List, Optional
from pathlib import Path
from collections import defaultdict

In [72]:
import pandas as pd
import numpy as np

In [402]:
dataframes = sorted(Path('../results').glob('*.csv'))

In [403]:
key_columns = ['제목', '방송 시작']
columns = ['제목', '방송 기간', '방송 횟수', '장르', '채널', '제작사', '극본', '출연자', '방송 시작']

In [404]:
frames = {
    data.stem: pd.read_csv(str(data))
    for data in dataframes
}

for frame in frames.values():
    assert (frame.columns == columns).all()

In [405]:
total = defaultdict(set)

In [406]:
for key, frame in frames.items():
    for _, args in frame[key_columns].iterrows():
        total[tuple(args.values)].add(key)

## Merge multiple domain dataframes

In [407]:
def merge(values: List[str]) -> str:
    """Simple merge return maximum length data
    """
    def valuable(value: np.ndarray) -> int:
        item = value.item()
        if isinstance(item, str):
            return len(item)
        return 0
    
    index = np.array(tuple(map(valuable, values))).argmax()
    
    return values[index].item()

In [408]:
table = pd.DataFrame([], columns=columns)

In [409]:
for key, domains in total.items():
    if len(domains) > 1:
        table = table.append(pd.Series([merge([frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)][column].values for domain in domains]) for column in columns], index=columns), ignore_index=True)
    else:
        domain, *_ = domains
        table = table.append(frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)])

In [413]:
print(table.shape)
table.head()

(1912, 9)


,제목,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자,방송 시작
0,비밀의숲,2017년 6월 10일 ~ 2017년 7월 30일,16부작 + 스페셜 1부작,"드라마, 스릴러, 범죄",tvN,"민현일, 고병철, 이성진\n 씨그널 엔터테인먼트 그룹 IOK 미디어",이수연(작가) 이수연(작가),"조승우, 배두나, 이준혁, 유재명, 신혜선 外",2017-06-10 00:00:00
1,붉은달푸른해,2018년 11월 21일 ~ 2019년 1월 16일,32부작 + 스페셜 1부작,미스터리 스릴러,MBC TV,메가몬스터]라는 CJ E&M 계열회사였으나 2017년 카카오에 매각되었다. ] ...,도현정,"김선아, 이이경, 남규리, [[엔 (가수)",2018-11-21 00:00:00
2,시를잊은그대에게,2018년 3월 26일 ~ 2018년 5월 15일,16부작,[[텔레비전 드라마,tvN,tvN,"명수현, 백선우, 최보림 (이상 1~16회)서동범 (15~16회)","이유비, [[이준혁 (1984년)",2018-03-26 00:00:00
3,메리대구공방전,2007년 5월 16일 ~ 2007년 7월 5일,16부작,"드라마, 로맨틱 코미디",문화방송,NaN,[[김인영 (작가),"지현우, 이하나, 왕빛나, 이민우(배우) 外",2007-05-16 00:00:00
4,안녕하세요하느님,2006년 1월 9일 ~ 2006년 2월 28일,16부작,"멜로, 드라마",KBS 2TV,"김종학프로덕션, 포도나무프로덕션",강은경,"유건, 김옥빈, 이종혁(배우) 外",2006-01-09 00:00:00


In [414]:
table.to_csv('../results/results.csv', index=None)